In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
df_energydata = pd.read_csv("/content/drive/My Drive/Colab Notebooks/energydata.csv")

In [11]:
df_energydata.shape

(19735, 29)

In [11]:
df_traindata, df_testdata = train_test_split(df_energydata, test_size=0.3)

print(df_traindata.shape)
print(df_testdata.shape)

(13814, 29)
(5921, 29)


In [12]:
X_test = df_testdata.iloc[:,3:27]  
Y_test = df_testdata.iloc[:,28] 
print(X_test.shape)
print(Y_test.shape)

(5921, 24)
(5921,)


In [0]:
ensemble = 100
frac = 0.7

predictions_total = np.zeros(5921, dtype=float)

for i in range(ensemble):
    print("number of iteration:", i)
    print("predictions_total", predictions_total)

    # Sample randomly the train data
    Traindata = df_traindata.sample(frac=frac)
    X_train = Traindata.iloc[:,3:27]  
    Y_train = Traindata.iloc[:,28] 
    

    ############################################################
    
    model = Sequential()
    # Adding the input layer and the first hidden layer
    model.add(Dense(units=36, kernel_initializer = 'normal', activation = 'relu', input_dim = 24))

    # Adding the second hidden layer
    model.add(Dense(units = 24, kernel_initializer = 'normal', activation = 'relu'))
    
    # Adding the third hidden layer
    model.add(Dense(units = 16, kernel_initializer = 'normal', activation = 'relu'))

    # Adding the output layer
    model.add(Dense(units = 1, kernel_initializer = 'normal', activation = 'relu'))

    # Compiling the ANN
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    model.compile(loss='mse', optimizer=adam, metrics=['mean_squared_error'])
    # Fitting the ANN to the Training set

    model.fit(X_train, Y_train, batch_size = 16, epochs = 100)

    ############################################################
    
    # We use predict() to predict our values
    model_predictions = model.predict(X_test)
    
    model_predictions = model_predictions.flatten()
    print("TEST MSE for individual model: ", mean_squared_error(Y_test, model_predictions))
    print("")
    print(model_predictions)
    print("")

    predictions_total = np.add(predictions_total, model_predictions)
 



number of iteration: 0
predictions_total [0. 0. 0. ... 0. 0. 0.]
Epoch 1/100
9670/9670 [==============================] - 5s 469us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 2/100
9670/9670 [==============================] - 3s 292us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 3/100
9670/9670 [==============================] - 3s 291us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 4/100
9670/9670 [==============================] - 3s 304us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 5/100
9670/9670 [==============================] - 3s 300us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 6/100
9670/9670 [==============================] - 3s 294us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 7/100
9670/9670 [==============================] - 3s 299us/step - loss: 845.6162 - mean_squared_error: 845.6162
Epoch 8/100
9670/9670 [==============================] - 3s 303us/step - loss: 845.6162 - mean_squared

In [0]:
# out of for...loop
print("")
predictions_total = predictions_total/ensemble
print("MSE after ensemble: ", mean_squared_error(np.array(Y_test), predictions_total))
print("")
print(predictions_total)